Projenin amacı, bir tavsiye sistemi uygulayarak şirketin artan satışlar elde etmesine yardımcı olmaktır.

Kayıp veri

▪ Maksimum satışları kullanarak en popüler ürünü bulmayı hesaplayın

hacim ve dolar miktarına göre

▪ Maksimum satın alma olan şirket

▪ Popülerliğe dayalı bir tavsiye sistemi oluşturun

▪ Matris ayrıştırmayı kullanarak bir tavsiye sistemi oluşturun

▪ Ürün açıklaması için kosinüs benzerliği

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv('PBL 5 recommendation data.csv',encoding='latin-1')

In [3]:
df.head()

,Customers.id,Customers.fname,Customers.lname,Customers.company,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,...,Products.google_shopping_label,Products.product_option,Products.size,Products.material,Products.arm_style,Products.leg_style,Products.seat_size,Products.family_id,Products.saved_status,Products.freight_cost
0,797,Christy,Dill,Company0,1426018724,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF61071,0.0,NaN
1,3,John,Smith,Company1,1386089139,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF02132,NaN,NaN
2,3,John,Smith,Company1,1386089139,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2 x Extra large,Nitrile,NaN,NaN,NaN,PF00342,0.0,NaN
3,4,James,Anderson,NaN,1386780263,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF04970,NaN,NaN
4,5,Abraham,Pollak,Company3,1386861599,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF03045,NaN,NaN


In [4]:
#df.columns.tolist()

In [5]:
#import datetime 
#import datetime as dt
#df['Customers.last_modified']=df['Customers.last_modified'].apply(lambda d: datetime.datetime.fromtimestamp(int(d)/1000).strftime('%Y-%m-%d %H:%M:%S'))

In [6]:
#Satılan ürünler
df["Order_Items.product_id"].nunique()

1753

In [7]:
df["Products.id"].nunique()

1710

In [8]:
df2=df[['Products.id','Products.name',"Products.price",'Products.shopping_brand','Order_Items.qty','Orders.subtotal','Customers.id','Orders.company']]
df2 #Yorum yapacağım sütunları seçtim.

,Products.id,Products.name,Products.price,Products.shopping_brand,Order_Items.qty,Orders.subtotal,Customers.id,Orders.company
0,2310.0,"Basic Steel Rollators,Green",57.64,MEDLINE,1,57.20,797,Company0
1,177.0,Urinary Drain Bags,10.29,MEDLINE,4,20.00,3,Company1
2,1.0,"SensiCare Nitrile Exam Gloves,Blue,XX-Large",68.78,MEDLINE,1,68.78,3,Company1
3,983.0,Basket for 2-Button Walkers,19.56,MEDLINE,1,19.56,4,NaN
4,991.0,TENS 3000 Analog Unit,36.05,ROSCOE MEDICAL INC,1,36.05,5,Company3
...,...,...,...,...,...,...,...,...
4189,1842.0,"Emesis Bags,Blue,36.000 OZ",23.65,MEDLINE,1,14.85,3730,NaN
4190,782.0,Bed Assist Bar,69.99,MEDLINE,1,39.99,3732,NaN
4191,2707.0,Spitfire EX Compact Travel Power Mobility Scoo...,1099.00,Drive Medical,1,1099.00,3733,NaN
4192,993.0,"Standard Bariatric Heavy Duty Rollator,Burgund...",129.99,MEDLINE,1,129.99,3735,NaN


In [9]:
items=df2.groupby('Products.id').sum()
items

,Products.price,Order_Items.qty,Orders.subtotal,Customers.id
Products.id,,,,
1.0,68.78,1,68.78,3
19.0,91.08,6,273.24,1577
20.0,200.20,2,397.76,3897
22.0,29.00,1,46.44,2215
30.0,114.87,5,189.56,3155
...,...,...,...,...
25612.0,19.80,2,376.23,3399
25694.0,22.48,1,22.48,3330
25908.0,52.40,1,52.40,3468


In [10]:
items[items['Orders.subtotal']==items['Orders.subtotal'].max()] #en çok harcama yapılan ürün->1846

,Products.price,Order_Items.qty,Orders.subtotal,Customers.id
Products.id,,,,
1846.0,622.98,396,13781.12,21864


In [11]:
items[items['Order_Items.qty']==items['Order_Items.qty'].max()] #en çok alınan ürün de 1846

,Products.price,Order_Items.qty,Orders.subtotal,Customers.id
Products.id,,,,
1846.0,622.98,396,13781.12,21864


In [12]:
df2[df2['Products.id']==1846.0] #En çok tercih edilen ürün 1846.0 ve MEDLINE markalıdır. (en popüler ürün)

,Products.id,Products.name,Products.price,Products.shopping_brand,Order_Items.qty,Orders.subtotal,Customers.id,Orders.company
846,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,10,346.10,699,Company59
847,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,11,380.71,699,Company59
848,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,20,692.20,699,Company59
849,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,10,346.10,699,Company59
850,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,30,1038.30,699,Company59
1553,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,2,144.78,1305,Company118
1555,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,1,34.61,1305,Company118
1597,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,30,1038.30,1352,Company59
1598,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,30,1038.30,1352,Company59
1599,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,30,1038.30,1352,Company59


In [13]:
company=df2.groupby('Orders.company').sum()
company

,Products.id,Products.price,Order_Items.qty,Orders.subtotal,Customers.id
Orders.company,,,,,
Bacharach Institute Rehab,16910.0,33.60,2,67.20,3221
Company0,2310.0,57.64,1,57.20,797
Company1,178.0,79.07,5,88.78,6
Company10,13016.0,974.30,6,952.91,6614
Company100,0.0,0.00,1,40.60,1134
...,...,...,...,...,...
Company99,1989.0,23.22,2,46.44,1121
"Goforth &amp; Assoc, Realtors",4138.0,288.66,2,398.18,924
Marian Family Dantel,7961.0,42.27,1,42.27,2849


In [14]:
company[company['Orders.subtotal']==company['Orders.subtotal'].max()] #Maksimum satın alma olan şirket-> Company59

,Products.id,Products.price,Order_Items.qty,Orders.subtotal,Customers.id
Orders.company,,,,,
Company59,27690.0,519.15,381,13186.41,17015


In [15]:
df2[df2['Orders.company']=="Company59"] #Company59 1846 id li MEDLINE markalı üründen satın almış.

,Products.id,Products.name,Products.price,Products.shopping_brand,Order_Items.qty,Orders.subtotal,Customers.id,Orders.company
846,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,10,346.10,699,Company59
847,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,11,380.71,699,Company59
848,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,20,692.20,699,Company59
849,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,10,346.10,699,Company59
850,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,30,1038.30,699,Company59
1597,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,30,1038.30,1352,Company59
1598,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,30,1038.30,1352,Company59
1599,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,30,1038.30,1352,Company59
1600,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,30,1038.30,1352,Company59
1601,1846.0,"Disposable Emergency Blanket,Not Applicable",34.61,MEDLINE,30,1038.30,1352,Company59


In [16]:
grouped=df.groupby('Products.name').agg({'Orders.subtotal':[np.size,np.sum,np.mean]})
grouped

Orders.subtotal          \
                                                              size     sum   
Products.name                                                                
1 Box Of Medline Commode Liners with Absorbent Pad             1.0   37.98   
1-Ply Blockade AngelStat Surgical Gowns,Ceil Bl...             1.0  235.05   
1/2" self-adhesive hook material, 25 yard dispe...             1.0   46.50   
1/2" self-adhesive loop material, 25 yard dispe...             1.0   46.50   
100% Cotton Unisex Reversible Hyperbaric Scrub ...             2.0  115.08   
...                                                            ...     ...   
Youth-Sized Walker Platform Attachment                         1.0   51.71   
Zip-Style Biohazard Specimen Bags,Clear                        1.0   48.41   
iWalker Euro Style Rollator                                    1.0  169.99   
sEMG - MyoTrac Home Trainer with Myoscan Active...             1.0  520.45   
sEMG - single use uni-gel electrodes only, case...             1.0  520.45   

                                                            
                                                      mean  
Products.name                                               
1 Box Of Medline Commode Liners with Absorbent Pad   37.98  
1-Ply Blockade AngelStat Surgical Gowns,Ceil Bl...  235.05  
1/2" self-adhesive hook material, 25 yard dispe...   46.50  
1/2" self-adhesive loop material, 25 yard dispe...   46.50  
100% Cotton Unisex Reversible Hyperbaric Scrub ...   57.54  
...                                                    ...  
Youth-Sized Walker Platform Attachment               51.71  
Zip-Style Biohazard Specimen Bags,Clear              48.41  
iWalker Euro Style Rollator                         169.99  
sEMG - MyoTrac Home Trainer with Myoscan Active...  520.45  
sEMG - single use uni-gel electrodes only, case...  520.45  

[1391 rows x 3 columns]

In [17]:
popular_items=grouped.sort_values(('Orders.subtotal','mean'),ascending=False)
popular_items.head() 
# 'mean' üzerinden ortalama olarak herkesin alımını incelememiz gerekiyor.

Orders.subtotal           \
                                                              size      sum   
Products.name                                                                 
Baseline&reg; Dynamometer - Smedley Spring - Ad...             1.0  3000.00   
Sanitary Pads with Adhesive & Wings                            2.0  5628.56   
2-Ply Tissue/Poly Professional Towels,Not Appli...             2.0  5559.40   
First Class School Chair Dining Tray                           1.0  2375.59   
Bariatric Heavy Duty Canvas Sling                              1.0  2017.92   

                                                             
                                                       mean  
Products.name                                                
Baseline&reg; Dynamometer - Smedley Spring - Ad...  3000.00  
Sanitary Pads with Adhesive & Wings                 2814.28  
2-Ply Tissue/Poly Professional Towels,Not Appli...  2779.70  
First Class School Chair Dining Tray                2375.59  
Bariatric Heavy Duty Canvas Sling                   2017.92

In [18]:
grouped_sum=grouped['Orders.subtotal']['sum'].sum()
popular_items['percentage']=popular_items['Orders.subtotal','sum'].div(grouped_sum)*100
popular_items.head()

Orders.subtotal           \
                                                              size      sum   
Products.name                                                                 
Baseline&reg; Dynamometer - Smedley Spring - Ad...             1.0  3000.00   
Sanitary Pads with Adhesive & Wings                            2.0  5628.56   
2-Ply Tissue/Poly Professional Towels,Not Appli...             2.0  5559.40   
First Class School Chair Dining Tray                           1.0  2375.59   
Bariatric Heavy Duty Canvas Sling                              1.0  2017.92   

                                                            percentage  
                                                       mean             
Products.name                                                           
Baseline&reg; Dynamometer - Smedley Spring - Ad...  3000.00   0.567753  
Sanitary Pads with Adhesive & Wings                 2814.28   1.065211  
2-Ply Tissue/Poly Professional Towels,Not Appli...  2779.70   1.052122  
First Class School Chair Dining Tray                2375.59   0.449583  
Bariatric Heavy Duty Canvas Sling                   2017.92   0.381893

In [19]:
popular_items.sort_values(('percentage'),ascending=False) #Sırasıyla popüler ürünler: 

Orders.subtotal            \
                                                              size       sum   
Products.name                                                                  
Disposable Emergency Blanket,Not Applicable                   18.0  13781.12   
MoliCare Disposable Super Plus Briefs,27"-47"                 71.0  12660.68   
Aluminum Transport Chair with 12" Wheels,Red,F:...            67.0   8781.76   
Aluminum Transport Chair with 12" Wheels,Blue,F...            67.0   8667.58   
K1 Basic Extra-Wide Wheelchairs                               54.0   7149.68   
...                                                            ...       ...   
DawnMist Gel Toothpaste,0.850 OZ                               1.0      9.24   
Suture Removal Trays,Stainless Steel                           1.0      8.58   
Enema Sets,1500.0 ML                                           1.0      7.60   
Leg Bags with Twist Valve                                      1.0      7.28   
Male External Catheters,Medium                                 1.0      4.24   

                                                               percentage  
                                                          mean             
Products.name                                                              
Disposable Emergency Blanket,Not Applicable         765.617778   2.608091  
MoliCare Disposable Super Plus Briefs,27"-47"       178.319437   2.396047  
Aluminum Transport Chair with 12" Wheels,Red,F:...  131.071045   1.661957  
Aluminum Transport Chair with 12" Wheels,Blue,F...  129.366866   1.640348  
K1 Basic Extra-Wide Wheelchairs                     132.401481   1.353084  
...                                                        ...        ...  
DawnMist Gel Toothpaste,0.850 OZ                      9.240000   0.001749  
Suture Removal Trays,Stainless Steel                  8.580000   0.001624  
Enema Sets,1500.0 ML                                  7.600000   0.001438  
Leg Bags with Twist Valve                             7.280000   0.001378  
Male External Catheters,Medium                        4.240000   0.000802  

[1391 rows x 4 columns]

### Matrix Factorization

In [20]:
popItems=df.pivot_table(index=['Customers.id'],columns=['Products.name'],values='Orders.subtotal').fillna(0)
popItems.head()

Products.name,1 Box Of Medline Commode Liners with Absorbent Pad,"1-Ply Blockade AngelStat Surgical Gowns,Ceil Blue,Extra Large","1/2"" self-adhesive hook material, 25 yard dispenser box, white","1/2"" self-adhesive loop material, 25 yard dispenser box, white","100% Cotton Unisex Reversible Hyperbaric Scrub Tops,Misty Green,MZS","100% Cotton Unisex Reversible Tops,Ceil Blue,MHS","18"" Hamper Stands,Standard","2"" self-adhesive loop material, 10 yard dispenser box, tan","2"" self-adhesive loop material, 25 yard dispenser box, tan","2-Ply Tissue/Poly Professional Towels,Not Applicable",...,"Wrist and Forearm Splint with Abducted Thumb,Large","Wrist and Forearm Splint with Abducted Thumb,Medium","Wrist and Forearm Splint with Abducted Thumb,Small","Wrist and Forearm Splints,Medium","Youth Two-Button Folding Walkers with 5"" Wheels,5""",Youth-Sized Walker Platform Attachment,"Zip-Style Biohazard Specimen Bags,Clear",iWalker Euro Style Rollator,"sEMG - MyoTrac Home Trainer with Myoscan Active Sensor and extender cable, 1-channel","sEMG - single use uni-gel electrodes only, case of 100"
Customers.id,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
mpi_subtotal=popItems['Disposable Emergency Blanket,Not Applicable']
mpi_subtotal.value_counts() #Bunu satın alan müşterilerin subtotalleri(en çok ödeme yaptıkları)

0.000       2960
1038.300       1
560.682        1
415.320        1
89.695         1
Name: Disposable Emergency Blanket,Not Applicable, dtype: int64

In [22]:
similarity_with_other_items=popItems.corrwith(mpi_subtotal)
similarity_with_other_items=similarity_with_other_items.sort_values(ascending=False)
similarity_with_other_items.head(10)
#Başarısız.

Products.name
Disposable Emergency Blanket,Not Applicable                                            1.000000
Disposable Polypropylene Fitted Stretcher Sheets,Not Applicable                        0.070997
Optifoam Site Dressings,4"X4"                                                         -0.000566
Sterile Bulb Irrigation Syringe Trays                                                 -0.000566
MAXAR Work Belt - Industrial Lumbo-Sacral Support (Deluxe): IBS-3000, Medium, Black   -0.000566
Elevated Toilet Seat                                                                  -0.000566
CURAD Prosorb Rolled Gauze                                                            -0.000566
CURAD Cast Protectors,Adult                                                           -0.000566
Remedy Phytoplex Cleansing Body Lotion,8.000 OZ                                       -0.000566
Nitro Euro Style Walker Rollator, Hemi Height                                         -0.000566
dtype: float64

### Cosine Similarity

Kaç müşteri kaç item sipariş etti inceleyelim...

In [23]:
df_list = df.groupby('Products.id')['Customers.id'].apply(list)

In [24]:
df_final = pd.DataFrame(columns = df['Customers.id'].unique(), index = df['Products.id'].unique()).fillna(0)

In [25]:
df_final.head() 

,797,3,4,5,7,8,10,11,12,13,...,3724,3725,3726,3728,3729,3730,3732,3733,3735,3736
2310.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
177.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
983.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
991.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df_final.index

Float64Index([ 2310.0,   177.0,     1.0,   983.0,   991.0,     nan,  1379.0,
                815.0,   795.0,  1430.0,
              ...
                251.0,   350.0, 13104.0,  5972.0,  5985.0, 14974.0, 16660.0,
              15649.0,  2707.0,   562.0],
             dtype='float64', length=1711)

In [27]:
df_final.drop(float('nan'), inplace = True)

In [28]:
for i in df_final.index:
    for j in df_list.loc[i]:
        df_final.loc[i][j]=1
#satın alımları doldurduk.

In [29]:
num_cust=df_final.sum(axis=1)

In [30]:
num_cust

2310.0     2
177.0      1
1.0        1
983.0      4
991.0      1
          ..
14974.0    1
16660.0    1
15649.0    1
2707.0     1
562.0      1
Length: 1710, dtype: int64

In [31]:
df_final.loc[1846].value_counts()
#1846 no'lu ürüne 4 kişi sahip.

0    3050
1       4
Name: 1846.0, dtype: int64

In [32]:
#similarity icin:
m=np.mean(num_cust)
s=np.std(num_cust)
stan_num_cust=num_cust.apply(lambda x:((x-m)/s))
stan_num_cust=stan_num_cust*-1

In [33]:
from scipy import spatial

In [34]:
def similarity(bought,other):
    spa = spatial.distance.cosine(list(df_final.loc[bought]),list(df_final.loc[other]))
    pop = stan_num_cust.loc[other]/100
    return(spa+pop)

In [35]:
similarity(1846,2310)

1.0002685546352197

In [36]:
def rec_list(bought):
    rec = pd.DataFrame(columns=['rec'],index=df_final.index)
    for i in rec.index:
        rec.loc[i,'rec']=similarity(bought,i)
    return(rec.sort_values(by='rec',ascending=True))

In [37]:
rec_list(1846)

,rec
1846.0,-0.00449029
1825.0,0.502648
1842.0,0.812294
910.0,0.845606
911.0,0.845606
...,...
9499.0,1.00265
9501.0,1.00265
9500.0,1.00265
13779.0,1.00265
